# 어텐션 메커니즘과 트랜스포머

순환 신경망(Recurrent Neural Networks, RNN)의 주요 단점 중 하나는 시퀀스 내 모든 단어가 결과에 동일한 영향을 미친다는 점입니다. 이는 이름 엔터티 인식(Named Entity Recognition)이나 기계 번역(Machine Translation)과 같은 시퀀스-투-시퀀스 작업에서 표준 LSTM 인코더-디코더 모델의 성능을 저하시키는 원인이 됩니다. 실제로 입력 시퀀스의 특정 단어는 다른 단어보다 순차적 출력에 더 큰 영향을 미치는 경우가 많습니다.

기계 번역과 같은 시퀀스-투-시퀀스 모델을 생각해봅시다. 이 모델은 두 개의 순환 신경망으로 구현되며, 하나의 네트워크(**인코더**)는 입력 시퀀스를 은닉 상태로 압축하고, 다른 네트워크(**디코더**)는 이 은닉 상태를 번역된 결과로 펼칩니다. 이 접근 방식의 문제는 네트워크의 최종 상태가 문장의 시작 부분을 기억하기 어렵다는 점이며, 이는 긴 문장에서 모델의 품질 저하를 초래합니다.

**어텐션 메커니즘**은 RNN의 각 출력 예측에 대해 각 입력 벡터의 맥락적 영향을 가중치로 부여하는 방법을 제공합니다. 이는 입력 RNN의 중간 상태와 출력 RNN 간에 단축 경로를 생성함으로써 구현됩니다. 이 방식으로 출력 심볼 $y_t$를 생성할 때, 서로 다른 가중치 계수 $\alpha_{t,i}$를 사용하여 모든 입력 은닉 상태 $h_i$를 고려합니다.

![어텐션 레이어가 추가된 인코더/디코더 모델](../../../../../lessons/5-NLP/18-Transformers/images/encoder-decoder-attention.png)
*[Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf)의 어텐션 메커니즘이 포함된 인코더-디코더 모델, [이 블로그 글](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)에서 인용됨*

어텐션 행렬 $\{\alpha_{i,j}\}$은 출력 시퀀스의 특정 단어를 생성하는 데 있어 특정 입력 단어가 얼마나 중요한지를 나타냅니다. 아래는 이러한 행렬의 예시입니다:

![Bahdanau - arviz.org에서 가져온 RNNsearch-50의 샘플 정렬](../../../../../lessons/5-NLP/18-Transformers/images/bahdanau-fig3.png)

*[Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf)에서 가져온 그림 (Fig.3)*

어텐션 메커니즘은 현재 또는 거의 현재의 자연어 처리(NLP) 분야에서 최첨단 기술의 많은 부분을 차지하고 있습니다. 그러나 어텐션을 추가하면 모델 매개변수의 수가 크게 증가하여 RNN에서 확장 문제를 초래합니다. RNN 확장의 주요 제약은 모델의 순환적 특성으로 인해 학습을 배치 및 병렬화하기 어렵다는 점입니다. RNN에서는 시퀀스의 각 요소를 순차적으로 처리해야 하므로 병렬화가 쉽지 않습니다.

어텐션 메커니즘의 채택과 이러한 제약은 오늘날 우리가 알고 사용하는 최첨단 트랜스포머 모델(BERT에서 OpenGPT3까지)의 탄생으로 이어졌습니다.

## 트랜스포머 모델

이전 예측의 맥락을 다음 평가 단계로 전달하는 대신, **트랜스포머 모델**은 **위치 인코딩(positional encodings)**과 어텐션을 사용하여 주어진 텍스트 창 내에서 입력의 맥락을 캡처합니다. 아래 이미지는 위치 인코딩과 어텐션이 주어진 창 내에서 맥락을 어떻게 캡처하는지 보여줍니다.

![트랜스포머 모델에서 평가가 수행되는 방식을 보여주는 애니메이션 GIF](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

각 입력 위치가 독립적으로 각 출력 위치에 매핑되기 때문에, 트랜스포머는 RNN보다 병렬화가 더 잘 이루어질 수 있으며, 이는 훨씬 더 크고 표현력이 뛰어난 언어 모델을 가능하게 합니다. 각 어텐션 헤드는 단어 간의 다양한 관계를 학습하는 데 사용될 수 있으며, 이는 자연어 처리 작업의 성능을 향상시킵니다.

**BERT**(Bidirectional Encoder Representations from Transformers)는 매우 큰 다층 트랜스포머 네트워크로, *BERT-base*는 12개 층, *BERT-large*는 24개 층으로 구성됩니다. 이 모델은 대규모 텍스트 데이터(WikiPedia + 책)를 사용하여 비지도 학습(문장에서 마스킹된 단어 예측)을 통해 먼저 사전 학습됩니다. 사전 학습 동안 모델은 상당한 수준의 언어 이해를 흡수하며, 이를 다른 데이터셋과 함께 미세 조정(fine-tuning)하여 활용할 수 있습니다. 이 과정을 **전이 학습(transfer learning)**이라고 합니다.

![http://jalammar.github.io/illustrated-bert/에서 가져온 그림](../../../../../lessons/5-NLP/18-Transformers/images/jalammarBERT-language-modeling-masked-lm.png)

BERT, DistilBERT, BigBird, OpenGPT3 등 다양한 트랜스포머 아키텍처 변형이 있으며, 이를 미세 조정할 수 있습니다. [HuggingFace 패키지](https://github.com/huggingface/)는 PyTorch를 사용하여 이러한 아키텍처 중 다수를 학습할 수 있는 저장소를 제공합니다.

## BERT를 사용한 텍스트 분류

이제 사전 학습된 BERT 모델을 사용하여 전통적인 작업인 시퀀스 분류를 해결하는 방법을 살펴보겠습니다. 우리는 원래의 AG News 데이터셋을 분류할 것입니다.

먼저 HuggingFace 라이브러리와 데이터셋을 로드해봅시다:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


사전 학습된 BERT 모델을 사용할 것이기 때문에 특정 토크나이저를 사용해야 합니다. 먼저, 사전 학습된 BERT 모델과 연결된 토크나이저를 로드하겠습니다.

HuggingFace 라이브러리는 사전 학습된 모델의 저장소를 포함하고 있으며, 모델 이름을 `from_pretrained` 함수의 인수로 지정하기만 하면 사용할 수 있습니다. 모델에 필요한 모든 바이너리 파일은 자동으로 다운로드됩니다.

하지만 때로는 직접 만든 모델을 로드해야 할 때가 있습니다. 이 경우 토크나이저의 매개변수, 모델 매개변수가 포함된 `config.json` 파일, 바이너리 가중치 등을 포함한 관련 파일이 있는 디렉토리를 지정할 수 있습니다.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer` 객체는 텍스트를 직접 인코딩하는 데 사용할 수 있는 `encode` 함수를 포함하고 있습니다:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

그렇다면, 훈련 중 데이터를 접근하기 위해 사용할 반복자를 만들어 봅시다. BERT는 자체 인코딩 함수를 사용하기 때문에 이전에 정의한 `padify`와 유사한 패딩 함수를 정의해야 합니다.


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

우리의 경우, `bert-base-uncased`라는 사전 학습된 BERT 모델을 사용할 것입니다. `BertForSequenceClassification` 패키지를 사용하여 모델을 로드해 봅시다. 이를 통해 우리의 모델이 분류를 위한 필요한 아키텍처를 이미 갖추고 있으며, 최종 분류기를 포함하고 있음을 보장합니다. 최종 분류기의 가중치가 초기화되지 않았으며 모델이 사전 학습을 필요로 한다는 경고 메시지를 보게 될 것입니다. 이는 완전히 괜찮습니다. 왜냐하면 바로 그것이 우리가 하려는 일이기 때문입니다!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

이제 훈련을 시작할 준비가 되었습니다! BERT는 이미 사전 학습이 완료된 모델이기 때문에 초기 가중치를 손상시키지 않기 위해 비교적 작은 학습률로 시작하는 것이 좋습니다.

모든 주요 작업은 `BertForSequenceClassification` 모델이 수행합니다. 훈련 데이터를 모델에 호출하면 입력 미니배치에 대해 손실과 네트워크 출력을 반환합니다. 우리는 손실을 매개변수 최적화에 사용하며 (`loss.backward()`는 역전파를 수행합니다), `out`은 `argmax`를 사용해 계산된 레이블 `labs`과 기대 레이블 `labels`을 비교하여 훈련 정확도를 계산하는 데 사용합니다.

훈련 과정을 제어하기 위해 여러 반복 동안 손실과 정확도를 누적하고, 이를 `report_freq` 훈련 주기마다 출력합니다.

이 훈련은 아마도 상당히 오랜 시간이 걸릴 가능성이 있으므로 반복 횟수를 제한합니다.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


BERT 분류를 사용하면 (특히 반복 횟수를 늘리고 충분히 기다리면) 꽤 좋은 정확도를 얻을 수 있다는 것을 알 수 있습니다! 이는 BERT가 이미 언어 구조를 꽤 잘 이해하고 있기 때문이며, 우리는 최종 분류기를 미세 조정하기만 하면 됩니다. 하지만 BERT는 큰 모델이기 때문에 전체 학습 과정이 오래 걸리고 상당한 계산 능력이 필요합니다! (GPU, 그리고 가능하면 여러 대가 필요합니다).

> **Note:** 우리의 예제에서는 가장 작은 사전 학습된 BERT 모델 중 하나를 사용하고 있습니다. 더 큰 모델들은 더 나은 결과를 낼 가능성이 높습니다.


## 모델 성능 평가

이제 테스트 데이터셋에서 모델의 성능을 평가할 수 있습니다. 평가 루프는 훈련 루프와 매우 유사하지만, `model.eval()`을 호출하여 모델을 평가 모드로 전환하는 것을 잊지 말아야 합니다.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## 주요 내용

이 단원에서는 **transformers** 라이브러리에서 사전 학습된 언어 모델을 가져와 텍스트 분류 작업에 쉽게 적용할 수 있는 방법을 살펴보았습니다. 마찬가지로, BERT 모델은 엔티티 추출, 질문 응답, 기타 NLP 작업에도 사용할 수 있습니다.

Transformer 모델은 현재 NLP 분야에서 최첨단 기술을 대표하며, 대부분의 경우 맞춤형 NLP 솔루션을 구현할 때 처음으로 실험을 시작해야 하는 솔루션입니다. 하지만, 이 모듈에서 논의된 순환 신경망의 기본 원리를 이해하는 것은 고급 신경 모델을 구축하려는 경우 매우 중요합니다.



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 권위 있는 출처로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 책임을 지지 않습니다.
